this notebook is responsible for updating new covid-19 dataset and saving on SQLite3 database.

In [1]:
import json
import pandas as pd
from tqdm import tqdm

fetch data from nytime repo.

In [48]:
covid19_data_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
# covid19_data_url = "https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports_us/12-18-2021.csv"

data = pd.read_csv(covid19_data_url)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 108, saw 2


In [49]:
data

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-21,Washington,deaths,0
2,2020-01-22,Washington,confirmed,1
3,2020-01-22,Washington,deaths,0
4,2020-01-23,Washington,confirmed,1
...,...,...,...,...
72807,2021-12-22,West Virginia,deaths,5242
72808,2021-12-22,Wisconsin,confirmed,1072497
72809,2021-12-22,Wisconsin,deaths,10894
72810,2021-12-22,Wyoming,confirmed,114062


In [32]:
data = data[['date', 'state', 'cases', 'deaths']]
data = data.rename(columns={"state": "states"})

In [33]:
data

,date,states,cases,deaths
0,2020-01-21,Washington,1,0
1,2020-01-22,Washington,1,0
2,2020-01-23,Washington,1,0
3,2020-01-24,Illinois,1,0
4,2020-01-24,Washington,1,0
...,...,...,...,...
36401,2021-12-22,Virginia,1034107,15224
36402,2021-12-22,Washington,812338,9799
36403,2021-12-22,West Virginia,317540,5242
36404,2021-12-22,Wisconsin,1072497,10894


table reshaping for visualization

In [34]:
data['number'] = data['cases']
data['cases'] = 'confirmed'

In [35]:
data

,date,states,cases,deaths,number
0,2020-01-21,Washington,confirmed,0,1
1,2020-01-22,Washington,confirmed,0,1
2,2020-01-23,Washington,confirmed,0,1
3,2020-01-24,Illinois,confirmed,0,1
4,2020-01-24,Washington,confirmed,0,1
...,...,...,...,...,...
36401,2021-12-22,Virginia,confirmed,15224,1034107
36402,2021-12-22,Washington,confirmed,9799,812338
36403,2021-12-22,West Virginia,confirmed,5242,317540
36404,2021-12-22,Wisconsin,confirmed,10894,1072497


In [36]:
len_of_current_data = len(data)
data_list = data.values.tolist()


In [37]:
for each in tqdm(data_list[:len_of_current_data]):
    data_list.append(
        [
            each[0], each[1], 'deaths', each[3], each[3]
        ]
    )

100%|██████████| 36406/36406 [00:00<00:00, 1573636.64it/s]


In [38]:
data = pd.DataFrame(data=data_list, columns=list(data.columns))

In [39]:
data

,date,states,cases,deaths,number
0,2020-01-21,Washington,confirmed,0,1
1,2020-01-22,Washington,confirmed,0,1
2,2020-01-23,Washington,confirmed,0,1
3,2020-01-24,Illinois,confirmed,0,1
4,2020-01-24,Washington,confirmed,0,1
...,...,...,...,...,...
72807,2021-12-22,Virginia,deaths,15224,15224
72808,2021-12-22,Washington,deaths,9799,9799
72809,2021-12-22,West Virginia,deaths,5242,5242
72810,2021-12-22,Wisconsin,deaths,10894,10894


In [40]:
new_col = list(data.columns)
new_col.remove('deaths')

In [41]:
data = data[new_col]
data

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-22,Washington,confirmed,1
2,2020-01-23,Washington,confirmed,1
3,2020-01-24,Illinois,confirmed,1
4,2020-01-24,Washington,confirmed,1
...,...,...,...,...
72807,2021-12-22,Virginia,deaths,15224
72808,2021-12-22,Washington,deaths,9799
72809,2021-12-22,West Virginia,deaths,5242
72810,2021-12-22,Wisconsin,deaths,10894


In [42]:
data = data.sort_values(['date', 'states', 'cases']).reset_index(drop=True)
data 

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-21,Washington,deaths,0
2,2020-01-22,Washington,confirmed,1
3,2020-01-22,Washington,deaths,0
4,2020-01-23,Washington,confirmed,1
...,...,...,...,...
72807,2021-12-22,West Virginia,deaths,5242
72808,2021-12-22,Wisconsin,confirmed,1072497
72809,2021-12-22,Wisconsin,deaths,10894
72810,2021-12-22,Wyoming,confirmed,114062


save the dataset into SQLite3 databse

In [43]:
import sqlite3
import os

In [44]:
if not os.path.exists('./dataset/database/covid19'):
    os.makedirs('./dataset/database/covid19')

In [45]:
conn = sqlite3.connect('./dataset/database/covid19/covid19.sqlite')

In [46]:
data.to_sql('us_states', conn, if_exists='replace', index=False)

In [47]:
pd.read_sql('select * from us_states', conn)

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-21,Washington,deaths,0
2,2020-01-22,Washington,confirmed,1
3,2020-01-22,Washington,deaths,0
4,2020-01-23,Washington,confirmed,1
...,...,...,...,...
72807,2021-12-22,West Virginia,deaths,5242
72808,2021-12-22,Wisconsin,confirmed,1072497
72809,2021-12-22,Wisconsin,deaths,10894
72810,2021-12-22,Wyoming,confirmed,114062


# Example Visualizations